# Partition function approach

## Goals
In this notebook, we will 

1. Justify the matricies to use for a/c transformations
2. Construct a set of transformations that cover the entire space of the cell shape (at least for bcc/omega/hcp Ti)
3. Create a workflow to run the structures
4. Analyze the results (plotting a 0K surface and finding the minima)
5. Construct a workflow to calculation vibrational properties of the selected minima
6. Explore how to extend this method (specifically deformations) to any pure element for all the structures of that element in the Materials Project

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from pymatgen import Lattice, MPRester

## 1. Justify the matricies for a/c transformations

We need to find the set of matricies that lets us adjust our a/c ratios, but maintain symmetry. The approach is to create hexagonal lattices with pymatgen, get the matrices and do an inversion and dot product to get the transformation matrix. For the `a` transformation:

In [ ]:
hcp = Lattice.hexagonal(2,4)
a_deformed_hcp = Lattice.hexagonal(2.2 ,4) # 10% increased a hcp lattice
transformation_matrix = np.linalg.inv(hcp.matrix).dot(a_deformed_hcp.matrix)
print(transformation_matrix)

Taking the very small numbers as 0, we can see that the transformation matrix for increasing `a` by 10% is:
```
[[  1.1  0.0  0.0]
 [  0.0  1.1  0.0]
 [  0.0  0.0  1.0]]
```
or more generally,
```
[[  1+x  0.0  0.0]
 [  0.0  1+x  0.0]
 [  0.0  0.0  1.0]]
```
where `x` is the deformation fraction.

Let's do the same for `c`:

In [ ]:
c_deformed_hcp = Lattice.hexagonal(2 ,4.4) # 10% increased c hcp lattice
transformation_matrix = np.linalg.inv(hcp.matrix).dot(c_deformed_hcp.matrix)
print(transformation_matrix)

We can generalize this (with the rounding) to a a similar
```
[[  1.0  0.0  0.0]
 [  0.0  1.0  0.0]
 [  0.0  0.0  1+x]]
```

## 2. Construct a set of transformations that cover the entire space of the cell shape

Now that we know how to construct matricies that deform `a` and `c` alone, we can create a list of matrices that form a plane mesh of deformations over the entire `a`-`c` space. 

In [ ]:
mpr = MPRester()
# mp-72 is P6/mmm
# mp-46 is P6_3/mmc
# mp-6985 is Fm-3m
# mp-73 is Im-3m
hcp_ti = mpr.get_structure_by_material_id('mp-46')
omega_ti = mpr.get_structure_by_material_id('mp-72')
bcc_ti = mpr.get_structure_by_material_id('mp-73')

First we should make sure we can get `a`-`c` limits for hcp/bcc. We'll make a supercell of bcc ([1,1,2]) to make the cell size comparable to the hcp (2 atoms in each). 

In [ ]:
print('HCP a: {:0.3f}\nHCP c: {:0.3f}'.format(hcp_ti.lattice.a, hcp_ti.lattice.c))

bcc_ti_ss = bcc_ti.copy()
bcc_ti_ss.make_supercell([1,1,2])
print('BCC a: {:0.3f}\nBCC c: {:0.3f}'.format(bcc_ti_ss.lattice.a, bcc_ti_ss.lattice.c))

So we need to get `a` and `b` from 2.939 to 2.816 (0.958 fraction so we should decrease by at least -4.2% deformation would to sufficently deform `a`). For `c`, the hcp cell would need to increase from 4.641 to 5.631 (1.213 fraction, so at least a +21.3% deformation is required).

Now to compare hcp and omega, it takes a little more effort to get comparable cell sizes. Supercells of [3,3,2] and [2,2,3] to get 24 atoms in each cell.

In [ ]:
hcp_ti_ss = hcp_ti.copy()
hcp_ti_ss.make_supercell([3,3,2])
print('HCP   a: {:0.3f}\nHCP   c: {:0.3f}'.format(hcp_ti_ss.lattice.a, hcp_ti_ss.lattice.c))

omega_ti_ss = omega_ti.copy()
omega_ti_ss.make_supercell([2,2,3])
print('Omega a: {:0.3f}\nOmega c: {:0.3f}'.format(omega_ti_ss.lattice.a, omega_ti_ss.lattice.c))

Similarly, we need to increase the `a` of HCP from 8.817 to 9.154 (+3.8%) and decrease the `c` from 9.282 to 8.486 (-8.6%). Thus the limits are conviently defined as:

### Deformation limits

The theoretical minimum limits, the proposed limits and proposed number of deformations are:

|   |  min |  max | |min | max | n  |
|:-:|:----:|:----:| |:---:|:--:|:---:|
| a | -4.2 |  3.8 | |-5  | 5  | 10  |
| c | -8.6 | 21.3 | |-10 | 25 | 20  |


## 3. Create a workflow to run the structures

We'll construct these deformations matricies, broadcast them to create a mesh of matricies and then create a workflow. Unlike before, we only need one workflow now to match all the cell sizes. 
# (**Is this true? What about ionic positions?**) 

In [ ]:
# construct matricies
a_min = -0.05; a_max = 0.05; a_n = 10
a_defos = np.array([np.array([[1+x,0,0],[0,1+x,0],[0,0,1]]) for x in np.linspace(a_min, a_max, a_n)])
c_min = -0.10; c_max = 0.25; c_n = 20
c_defos = np.array([np.array([[1,0,0],[0,1,0],[0,0,1+x]]) for x in np.linspace(c_min, c_max, c_n)])

# broacast the deformation arrays and flatten them
broadcasted_defos = a_defos*c_defos[:,np.newaxis]
defos = broadcasted_defos.reshape(200, 3, 3)
print('{} deformed structures will be calculated.'.format(len(defos)))

# create the workflow and load it in launchpad
from atomate.vasp.workflows.base.deformations import get_wf_deformations
from atomate.vasp.powerups import add_modify_incar, remove_custodian, add_tags
from fireworks import LaunchPad

API_KEY = None # If None, will use the one from your ~/.pmgrc.yaml. Otherwise string of API_KEY
LAUNCHPAD_FILE = '/Users/brandon/.fireworks-remote/my_launchpad.yaml' # If None, will try to autoload from FW_CONFIG_FILE environment variable. Otherwise string of file path
MP_ID = 'mp-46' # materials project id of the structure. mp-72 is the P6/mmm Ti
# mp-46 is P6_3/mmc
# mp-6985 is Fm-3m
# mp-73 is Im-3m

if API_KEY:
    mpr = MPRester(API_KEY)
else:
    mpr = MPRester()

if LAUNCHPAD_FILE:
    lpad = LaunchPad.from_file(LAUNCHPAD_FILE)
else:
    lpad = LaunchPad.auto_load()

# here the values in the linspace are the scaling factor for cell volume. E.g. -0.1 = 90%
# cubic_transformations = [np.eye(3)*((x+1)**(1.0/3.0)) for x in np.linspace(-0.1, 0.1, number_of_deformations)]

# construct the workflow
struct = hcp_ti
wf = get_wf_deformations(struct, defos, name="deformation", vasp_cmd=">>vasp_cmd<<", db_file=">>db_file<<")
wf = add_modify_incar(wf, modify_incar_params={"incar_update":{"SIGMA":0.2, "ISMEAR":1}}, fw_name_constraint='opt')
wf = add_tags(wf, ['partition_function', 'hcp_ti'])
lpad.add_wf(wf)

# Analyze the results

First we'll get all of the data from our database. Then we'll define a function to transform it to something plottable by matplotlib (`mesh()`). Then we'll plot up the data.

In [ ]:
from atomate.vasp.database import MMVaspDb
mmdb = MMVaspDb.from_db_file("/Users/brandon/.fireworks-remote/db.json")

a_list = []
c_list = []
energies = []

defo_task_ids = range(52, 252)
for task_id in defo_task_ids:
    task = mmdb.collection.find_one({"task_id":task_id})
    energies.append(task["output"]["energy"])
    a_list.append(task["output"]["structure"]["lattice"]["a"])
    c_list.append(task["output"]["structure"]["lattice"]["c"])

In [ ]:
def mesh(xs, ys, zs):
    """
    Takes 3 1D arrays and returns meshed xs and ys, and reshaped zs. 
    
    Shape is a tuple like in MPL
    
    Example xs=[1,2,1,2], ys=[0,0,1,1], zs=[1,2,3,4]"""
    xs_unique = np.array(sorted(list(set(xs))))
    ys_unique = np.array(sorted(list(set(ys))))
    meshed_x, meshed_y = np.meshgrid(xs_unique, ys_unique)
    # sort the zs into the correct indices
    meshed_z = np.zeros(meshed_x.shape)
    for x, y, z in zip(xs, ys, zs):
        j = np.argwhere(xs_unique == x)
        i = np.argwhere(ys_unique == y)
        meshed_z[i,j] = z
    return (meshed_x, meshed_y, meshed_z)


In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

a, c, energies_m = mesh(a_list, c_list, energies)

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(a, c, energies_m, cmap=cm.viridis)
m = cm.ScalarMappable(cmap=cm.viridis)
m.set_array(energies)
fig.colorbar(m, shrink=0.6, aspect=5)

## Analysis

It appears that we did not get the nice multiple minima just from changing the a/c ratio. This is what we might expect based on the work of Morten. We'd have to either 

1. relax the ionic positions at each deformation to the equilibrium structure and energy or
2. Do these plots for all the structures and combine them somehow

The workflow below will try the first (just for kicks). All we are doing is changing the `relax_deformed` parameter to true in the `get_wf_deformations` and then making sure that we apply Methfessel-Paxton smearing since we are doing a relaxation.

In [ ]:
struct = hcp_ti
wf = get_wf_deformations(struct, defos, name="deformation", vasp_cmd=">>vasp_cmd<<", db_file=">>db_file<<", relax_deformed=True)
wf = add_modify_incar(wf, modify_incar_params={"incar_update":{"SIGMA":0.2, "ISMEAR":1}})
wf = add_tags(wf, ['partition_function', 'hcp_ti', 'relax'])
lpad.add_wf(wf)

Now lets do the same analysis and plotting. The below proves that there was only one ionic step for each.

In [ ]:
a_list = []
c_list = []
energies = []

defo_task_ids = range(253, 453)
for task_id in defo_task_ids:
    task = mmdb.collection.find_one({"task_id":task_id})
    assert 'deformation' in task["task_label"]
    assert len(task['calcs_reversed'][0]['output']['ionic_steps']) == 1
    energies.append(task["output"]["energy"])
    a_list.append(task["output"]["structure"]["lattice"]["a"])
    c_list.append(task["output"]["structure"]["lattice"]["c"])

a, c, energies_m = mesh(a_list, c_list, energies)

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(a, c, energies_m, cmap=cm.viridis)
m = cm.ScalarMappable(cmap=cm.viridis)
m.set_array(energies)
fig.colorbar(m, shrink=0.6, aspect=5)

So the next step is to try to perturb them, or make surfaces for the other structures. We will do the second. To get the same range, we will redefine the deformations. The space of our energy surface is the following:

In [ ]:
print('Min a: {}'.format(min(a_list)))
print('Max a: {}'.format(max(a_list)))
print('Min c: {}'.format(min(c_list)))
print('Max c: {}'.format(max(c_list)))

We are not relaxing here. 

In [ ]:
# construct matricies
a_min = min(a_list)/bcc_ti.lattice.a-1; a_max = max(a_list)/bcc_ti.lattice.a-1; a_n = 10
a_defos = np.array([np.array([[1+x,0,0],[0,1+x,0],[0,0,1]]) for x in np.linspace(a_min, a_max, a_n)])
c_min = min(c_list)/bcc_ti.lattice.c-1; c_max = max(c_list)/bcc_ti.lattice.c-1; c_n = 20
c_defos = np.array([np.array([[1,0,0],[0,1,0],[0,0,1+x]]) for x in np.linspace(c_min, c_max, c_n)])

# broacast the deformation arrays and flatten them
broadcasted_defos = a_defos*c_defos[:,np.newaxis]
defos = broadcasted_defos.reshape(200, 3, 3)

struct = bcc_ti
wf = get_wf_deformations(struct, defos, name="bcc", vasp_cmd=">>vasp_cmd<<", db_file=">>db_file<<")
wf = add_modify_incar(wf, modify_incar_params={"incar_update":{"SIGMA":0.2, "ISMEAR":1}}, fw_name_constraint='opt')
wf = add_tags(wf, ['partition_function', 'bcc_ti'])
lpad.add_wf(wf)

In [ ]:
# construct matricies
a_min = min(a_list)/omega_ti.lattice.a-1; a_max = max(a_list)/omega_ti.lattice.a-1; a_n = 10
a_defos = np.array([np.array([[1+x,0,0],[0,1+x,0],[0,0,1]]) for x in np.linspace(a_min, a_max, a_n)])
c_min = min(c_list)/omega_ti.lattice.c-1; c_max = max(c_list)/omega_ti.lattice.c-1; c_n = 20
c_defos = np.array([np.array([[1,0,0],[0,1,0],[0,0,1+x]]) for x in np.linspace(c_min, c_max, c_n)])

# broacast the deformation arrays and flatten them
broadcasted_defos = a_defos*c_defos[:,np.newaxis]
defos = broadcasted_defos.reshape(200, 3, 3)

struct = omega_ti
wf = get_wf_deformations(struct, defos, name="omega", vasp_cmd=">>vasp_cmd<<", db_file=">>db_file<<")
wf = add_modify_incar(wf, modify_incar_params={"incar_update":{"SIGMA":0.2, "ISMEAR":1}}, fw_name_constraint='opt')
wf = add_tags(wf, ['partition_function', 'omega_ti'])
lpad.add_wf(wf)